#Лабораторная работа 1

#1. Установка библиотек

In [9]:
!pip install datasets pandas numpy scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [11]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/44/3c/d717024885424591d5376220b5e836c2d5293ce2011523c9de23ff7bf068/pip-25.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.1 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


#2. Импорт

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest, f_classif

#3. Датасет

In [2]:
dataset = load_dataset("imodels/credit-card")

df = pd.DataFrame(dataset["train"])
df.head()

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/24000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6000 [00:00<?, ? examples/s]

,limit_bal,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,...,education:2,education:3,education:4,education:5,education:6,marriage:0,marriage:1,marriage:2,marriage:3,default.payment.next.month
0,80000.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,75125.0,77353.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,30000.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,29242.0,29507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,180000.0,44.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,20916.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0
3,60000.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,58839.0,53235.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,130000.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,111587.0,112348.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


#4. Оставляем только числа

In [3]:
df = df.select_dtypes(include=np.number)
df = df.dropna()

#5. Удаление выбросов

In [4]:
iso = IsolationForest(contamination=0.05, random_state=42)

df["outlier"] = iso.fit_predict(df)

df_clean = df[df["outlier"] == 1].drop(columns=["outlier"])

print(df.shape)
print(df_clean.shape)

(24000, 35)
(22800, 34)


#6. Отбор признаков

In [5]:
target = "default.payment.next.month"

X = df_clean.drop(columns=[target])
y = df_clean[target]

selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print(selected_features)

Index(['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5'], dtype='str')
